# Merging two Data-sets
(It took me a long time and a lot of reading about how dask computes the task but i got it working.)
you will need npm, node.js and dask with pandas
https://docs.dask.org/en/latest/install.html

## First we define a localcluster and a client

In [1]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

## Some info about the client and all of our peons:

In [6]:
client

Client Scheduler: tcp://127.0.0.1:52807 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.51 GB


In [4]:
cluster.workers

{0: <Nanny: tcp://127.0.0.1:52838, threads: 1>,
 1: <Nanny: tcp://127.0.0.1:52835, threads: 1>,
 2: <Nanny: tcp://127.0.0.1:52829, threads: 1>,
 3: <Nanny: tcp://127.0.0.1:52832, threads: 1>}

## Now, here is where the fun begins

For some bizzare reason it took me almost a week to figure out dask.
I don't want to blame the library for it but the fact that dask's errors are vague does not help at all.
90% of the time i was geting a memory error and not much more to work with.

back to the task

## We need to read the tsv files to dataframes very simillar to those used by pandas
Make sure to type in the right name of the file

In [5]:
import dask.dataframe as dask

df1 = dask.read_csv("title.basics.full.tsv",sep='\t',dtype=str)
df1 = df1.set_index('tconst')

df2 = dask.read_csv("title.ratings.full.tsv",sep='\t',dtype=str)
df2 = df2.set_index('tconst')

In [7]:
df1 = client.persist(df1)

## Some lovely information about our datasets:

In [8]:
df1.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tconst,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
df2.head()

,averageRating,numVotes
tconst,,
tt0000001,5.6,1660
tt0000002,6.1,203
tt0000003,6.5,1373
tt0000004,6.2,123
tt0000005,6.2,2161


## This is just one of possible variations on joining data-sets

In [10]:
result = dask.merge(df1, df2, left_on='tconst', right_on='tconst').compute()

## Let's se what we got:

In [11]:
result.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
tconst,,,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.6,1660
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.1,203
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1373
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",6.2,123
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2161


look's about right, no column was lost and there is no duplicates to be seen

In [12]:
result.to_csv('all_videos.tsv', sep='\t')

And at long last we finnaly got to the finish line and it took little to no time compared to `merge_tsv_low_memory_use.py`